In [6]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [9]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import gradio as gr

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image).astype('float32')
    image = np.expand_dims(image, axis=0)
    image /= 255.0
    return image

def depreprocess_image(image):
    image = image.reshape((image.shape[0], image.shape[1], 3))
    image *= 255.0
    image = np.clip(image, 0, 255).astype('uint8')
    return image

def gram_matrix(tensor):
    channels = int(tensor.shape[-1])
    a = tf.reshape(tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

def content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - target))

def style_loss(base_style, gram_target):
    gram_base = gram_matrix(base_style)
    return tf.reduce_mean(tf.square(gram_base - gram_target))

content_weight = 1e4
style_weight = 1e-2

# VGG19'dan içerik ve stil katmanlarını çıkarıyoruz
def vgg_layers(layer_names):
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1']
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

vgg = vgg_layers(style_layers + content_layers)

def extract_features(image):
    outputs = vgg(image)
    style_outputs, content_outputs = (outputs[:num_style_layers], outputs[num_style_layers:])
    style_features = [gram_matrix(style_output) for style_output in style_outputs]
    content_features = content_outputs
    return style_features, content_features

# Stil transferi fonksiyonu
def run_style_transfer(content_image, style_image):
    content_image = load_and_preprocess_image(content_image)
    style_image = load_and_preprocess_image(style_image)

    generated_image = tf.Variable(content_image, dtype=tf.float32)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)

    epochs = 200
    results = []
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            generated_style_features, generated_content_features = extract_features(generated_image)
            target_style_features, _ = extract_features(style_image)
            _, target_content_features = extract_features(content_image)

            s_loss = tf.add_n([style_loss(generated_style, target_style)
                               for generated_style, target_style in zip(generated_style_features, target_style_features)])
            s_loss *= style_weight / num_style_layers

            c_loss = tf.add_n([content_loss(generated_content, target_content)
                               for generated_content, target_content in zip(generated_content_features, target_content_features)])
            c_loss *= content_weight / num_content_layers

            loss = s_loss + c_loss

        grads = tape.gradient(loss, generated_image)
        optimizer.apply_gradients([(grads, generated_image)])

        if epoch % 10 == 0:
            generated_image_numpy = generated_image.numpy()
            result_image = depreprocess_image(generated_image_numpy[0])
            plt.imshow(result_image)
            plt.title(f"Epoch {epoch}, Loss: {loss.numpy()}")
            plt.show()
            results.append((result_image, f"Epoch {epoch}, Loss: {loss.numpy()}"))

    return results[-1] if results else None

# Gradio arayüz fonksiyonu
def gradio_interface(content_image, style_image):
    result_image, loss_text = run_style_transfer(content_image, style_image)
    return result_image, loss_text

# Gradio arayüzü
interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Image(type="filepath", label='İçerik Resmi'),
        gr.Image(type="filepath", label='Stil Resmi')
    ],
    outputs=[
        gr.Image(type="filepath", label='Sonuç Resmi'),
        gr.Textbox(label='Kayıp')
    ],
    title="Neural Style Transfer",
    description="İçerik ve stil resimlerini yükleyip stil transferi işlemini başlatın."
)


# Arayüzü başlatıyoruz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d4b30bf93010d23d2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
